In [1]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="******",
  database="loan_default"
)

In [2]:
mycursor = mydb.cursor()

In [20]:
# Define schema for dimension tables
credit_info_schema = """
CREATE TABLE IF NOT EXISTS Credit_Info_Dim (
    Credit_Info_ID VARCHAR(255) PRIMARY KEY,
    ID INTEGER,
    Credit_Worthiness VARCHAR(255),
    Open_Credit VARCHAR(255),
    Credit_Type VARCHAR(255),
    Co_Applicant_Credit_Type VARCHAR(255)
);
"""

demographics_schema = """
CREATE TABLE IF NOT EXISTS Demographics_Dim (
    Demographics_ID VARCHAR(255) PRIMARY KEY,
    ID INTEGER,
    Year INTEGER,
    Gender VARCHAR(255),
    Age VARCHAR(255)
);
"""

loan_details_schema = """
CREATE TABLE IF NOT EXISTS Loan_Details_Dim (
    Loan_Details_ID VARCHAR(255) PRIMARY KEY,
    ID INTEGER,
    Loan_Limit VARCHAR(255),
    Approv_in_adv VARCHAR(255),
    Loan_Type VARCHAR(255),
    Loan_Purpose VARCHAR(255),
    Occupancy_Type VARCHAR(255),
    Total_Units VARCHAR(255)
);
"""

other_details_schema = """
CREATE TABLE IF NOT EXISTS Other_Details_Dim (
    Other_Details_ID VARCHAR(255) PRIMARY KEY,
    ID INTEGER,
    Business_or_Commercial VARCHAR(255),
    Submission_of_Application VARCHAR(255),
    Region VARCHAR(255),
    Security_Type VARCHAR(255)
);
"""

# Define schema for fact table
loan_fact_schema = """
CREATE TABLE IF NOT EXISTS Loan_Fact (
    ID INTEGER PRIMARY KEY,
    Demographics_ID VARCHAR(255),
    Loan_Details_ID VARCHAR(255),
    Credit_Info_ID VARCHAR(255),
    Other_Details_ID VARCHAR(255),
    Income FLOAT,
    LTV FLOAT,
    DTIR1 FLOAT,
    Loan_Amount FLOAT,
    Rate_of_Interest FLOAT,
    Term INTEGER,
    Property_Value FLOAT,
    Credit_Score INTEGER,
    Status INTEGER,
    FOREIGN KEY (Demographics_ID) REFERENCES Demographics_Dim(Demographics_ID),
    FOREIGN KEY (Loan_Details_ID) REFERENCES Loan_Details_Dim(Loan_Details_ID),
    FOREIGN KEY (Credit_Info_ID) REFERENCES Credit_Info_Dim(Credit_Info_ID),
    FOREIGN KEY (Other_Details_ID) REFERENCES Other_Details_Dim(Other_Details_ID)
);
"""

# Execute SQL queries to create tables
mycursor.execute(credit_info_schema)
mycursor.execute(demographics_schema)
mycursor.execute(loan_details_schema)
mycursor.execute(other_details_schema)
mycursor.execute(loan_fact_schema)

# Commit the changes
mydb.commit()


In [4]:
%pip install sqlalchemy

                                              0.0/2.1 MB ? eta -:--:--
     -                                        0.1/2.1 MB 1.7 MB/s eta 0:00:02
     ---                                      0.2/2.1 MB 2.4 MB/s eta 0:00:01
     -----                                    0.3/2.1 MB 2.1 MB/s eta 0:00:01
     --------                                 0.4/2.1 MB 2.4 MB/s eta 0:00:01
     ------------                             0.7/2.1 MB 3.0 MB/s eta 0:00:01
     ----------------                         0.9/2.1 MB 3.3 MB/s eta 0:00:01
     -----------------------                  1.2/2.1 MB 3.9 MB/s eta 0:00:01
     -----------------------------            1.5/2.1 MB 4.2 MB/s eta 0:00:01
     -----------------------------------      1.8/2.1 MB 4.5 MB/s eta 0:00:01
     ---------------------------------------  2.1/2.1 MB 4.5 MB/s eta 0:00:01
     ---------------------------------------- 2.1/2.1 MB 4.4 MB/s eta 0:00:00
                                              0.0/292.8 kB ? eta -:--:-


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from sqlalchemy import create_engine

In [6]:
%pip install pandas
import pandas as pd

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# Read dimension tables from CSV files
creditinfo_dim_df = pd.read_csv(r'C:\Users\Narendra\Desktop\DS\RISK_pyspark_vs\Credit_Info_Dim.csv')
demographic_dim_df = pd.read_csv(r'C:\Users\Narendra\Desktop\DS\RISK_pyspark_vs\Demographic_Dimension.csv')
loanapp_dim_df = pd.read_csv(r'C:\Users\Narendra\Desktop\DS\RISK_pyspark_vs\Loan_Application_Dimension.csv')
otherdetail_dim_df = pd.read_csv(r'C:\Users\Narendra\Desktop\DS\RISK_pyspark_vs\Other_Details_Dim.csv')

# Read fact table from CSV file
loan_fact_df = pd.read_csv(r'C:\Users\Narendra\Desktop\DS\RISK_pyspark_vs\Loan_Fact_Table.csv')

In [22]:
loan_fact_df.head(5)

,ID,Credit_Info_ID,Demographics_ID,Loan_Details_ID,Other_Details_ID,loan_amount,rate_of_interest,term,property_value,income,Credit_Score,LTV,Status,dtir1
0,24890,C1,d1,L1,O1,116500,NaN,360.0,118000.0,1740.0,758,98.728814,1,45.0
1,24891,C2,d2,L2,O2,206500,NaN,360.0,NaN,4980.0,552,NaN,1,NaN
2,24892,C3,d3,L3,O3,406500,4.56,360.0,508000.0,9480.0,834,80.019685,0,46.0
3,24893,C4,d4,L4,O4,456500,4.25,360.0,658000.0,11880.0,587,69.376900,0,42.0
4,24894,C5,d5,L5,O5,696500,4.00,360.0,758000.0,10440.0,602,91.886544,0,39.0


In [15]:
from sqlalchemy import create_engine

# Step 2: Create a SQLAlchemy engine to connect to the MySQL database
engine = create_engine("mysql+mysqlconnector://root:************/loan_default")

In [16]:
# Step 3: Convert the Pandas DataFrame to a format for MySQL table insertion

# Convert creditinfo_dim_df
creditinfo_dim_df.to_sql('Credit_Info_Dim', con=engine, if_exists='append', index=False)

# Convert demographic_dim_df
demographic_dim_df.to_sql('Demographics_Dim', con=engine, if_exists='append', index=False)

# Convert loanapp_dim_df
loanapp_dim_df.to_sql('Loan_Details_Dim', con=engine, if_exists='append', index=False)

# Convert otherdetail_dim_df
otherdetail_dim_df.to_sql('Other_Details_Dim', con=engine, if_exists='append', index=False)

# Convert loan_fact_df
loan_fact_df.to_sql('Loan_Fact', con=engine, if_exists='append', index=False)

C:\Users\Narendra\AppData\Local\Temp\ipykernel_6688\2161398014.py:4: UserWarning: The provided table name 'Credit_Info_Dim' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  creditinfo_dim_df.to_sql('Credit_Info_Dim', con=engine, if_exists='append', index=False)
C:\Users\Narendra\AppData\Local\Temp\ipykernel_6688\2161398014.py:7: UserWarning: The provided table name 'Demographics_Dim' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  demographic_dim_df.to_sql('Demographics_Dim', con=engine, if_exists='append', index=False)
C:\Users\Narendra\AppData\Local\Temp\ipykernel_6688\2161398014.py:10: UserWarning: The provided table name 'Loan_Details_Dim' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case tabl

148670

In [17]:
# Execute the SQL query to join tables
mycursor.execute("""
    CREATE TABLE IF NOT EXISTS risk_join AS
    SELECT
        f.ID,
        d.Year,
        d.Gender,
        d.Age,
        f.Income,
        f.LTV,
        f.DTIR1,
        ld.Loan_Limit,
        ld.Approv_in_adv,
        ld.Loan_Type,
        ld.Loan_Purpose,
        ld.Occupancy_Type,
        ld.Total_Units,
        ci.Credit_Worthiness,
        ci.Open_Credit,
        ci.Credit_Type,
        ci.Co_Applicant_Credit_Type,
        od.Business_or_Commercial,
        od.Submission_of_Application,
        od.Region,
        od.Security_Type,
        f.Status
    FROM
        Loan_Fact f
    JOIN Demographics_Dim d ON f.ID = d.ID
    JOIN Loan_Details_Dim ld ON f.ID = ld.ID
    JOIN Credit_Info_Dim ci ON f.ID = ci.ID
    JOIN Other_Details_Dim od ON f.ID = od.ID
""")

In [23]:
# Close the cursor and database connection
mycursor.close()
mydb.close()